<a href="https://colab.research.google.com/github/tammata2811/blockchain-project-2-IPR-right-for-artists-and-content-creator-in-VN/blob/main/Intellectual_Property_Rights_for_registeration%2C_divide_copyright_using_smart_contract%2C_license_manager%2C_Simulate_NFT_Marketplace_to_sell_works_without_third_parties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Intellectual Property Rights for registeration, divide copyright using smart contract, license manager, Simulate NFT Marketplace to sell works without third parties

####1.Generate SHA256 Hash Function – used to generate work ID

In [1]:
import hashlib
import time

def hash_function(input_str: str) -> str:
    return hashlib.sha256(input_str.encode()).hexdigest()


####2. Declare IP Asset and Transaction classes

In [2]:
class IPAsset:
    def __init__(self, owner, title, content_type, creation_date, authority):
        self.id = hash_function(owner + title)
        self.owner = owner
        self.title = title
        self.content_type = content_type
        self.creation_date = creation_date
        self.authority = authority

class Transaction:
    def __init__(self, from_owner, to_owner, ip_id):
        self.from_owner = from_owner
        self.to_owner = to_owner
        self.ip_id = ip_id
        self.timestamp = time.time()


####3. IPChain Management Layer

In [3]:
class IPChain:
    def __init__(self):
        self.assets = {}
        self.transactions = {}

    def register_asset(self, owner, title, content_type, creation_date, authority):
        asset_id = hash_function(owner + title)
        if asset_id in self.assets:
            print(f"[!] Tác phẩm đã tồn tại.")
            return None
        asset = IPAsset(owner, title, content_type, creation_date, authority)
        self.assets[asset.id] = asset
        print(f"[+] Đăng ký thành công: {title} bởi {owner}")
        return asset.id

    def transfer_rights(self, from_owner, to_owner, ip_id):
        asset = self.assets.get(ip_id)
        if not asset or asset.owner != from_owner:
            print("[!] Giao dịch không hợp lệ")
            return False
        asset.owner = to_owner
        self.transactions.setdefault(ip_id, []).append(Transaction(from_owner, to_owner, ip_id))
        print(f"[+] Chuyển quyền cho {to_owner} thành công.")
        return True

    def print_assets(self):
        print("\n🔐 Tác phẩm đã đăng ký:")
        for asset in self.assets.values():
            print(f"  - {asset.title} | Chủ: {asset.owner} | ID: {asset.id[:10]}...")


####4.Divide copyright using Smart Contract

In [4]:
class SmartContractRoyalty:
    def __init__(self, ip_id, contributors_shares):
        self.ip_id = ip_id
        self.shares = contributors_shares
        self.payments = []

    def distribute_payment(self, total_amount):
        print(f"\n💸 Phân phối {total_amount} VND cho tác phẩm {self.ip_id}")
        for contributor, percent in self.shares.items():
            amount = total_amount * percent / 100
            print(f"  -> {contributor} nhận {amount:.2f} VND ({percent}%)")
            self.payments.append({
                "contributor": contributor,
                "amount": amount
            })


####5.License Manager

In [5]:
class License:
    def __init__(self, user, ip_id, license_type, expiry_date):
        self.user = user
        self.ip_id = ip_id
        self.license_type = license_type
        self.expiry_date = expiry_date

class LicenseManager:
    def __init__(self):
        self.licenses = []

    def grant_license(self, user, ip_id, license_type, expiry_date):
        license_obj = License(user, ip_id, license_type, expiry_date)
        self.licenses.append(license_obj)
        print(f"[+] Cấp quyền {license_type} cho {user} tới {expiry_date} với tác phẩm {ip_id}")

    def check_license(self, user, ip_id):
        for l in self.licenses:
            if l.user == user and l.ip_id == ip_id:
                print(f"✔ {user} có quyền '{l.license_type}' với {ip_id} tới {l.expiry_date}")
                return
        print(f"✘ {user} không có quyền với {ip_id}")


####6. Simulate NFT Marketplace to sell works without third parties

In [6]:
class NFTMarketplace:
    def __init__(self):
        self.listings = {}

    def list_asset(self, asset_id, price):
        self.listings[asset_id] = price
        print(f"[+] Tác phẩm {asset_id[:10]}... đã được niêm yết với giá {price} VND")

    def buy_asset(self, buyer, ip_chain: IPChain, asset_id):
        if asset_id not in self.listings:
            print("[!] Tác phẩm không được niêm yết.")
            return
        asset = ip_chain.assets.get(asset_id)
        if not asset:
            print("[!] Tác phẩm không tồn tại.")
            return
        price = self.listings.pop(asset_id)
        ip_chain.transfer_rights(asset.owner, buyer, asset_id)
        print(f"💰 {buyer} đã mua tác phẩm với giá {price} VND")


####7.Demo

In [11]:
# Create an instance of IPChain
ip_chain = IPChain()
license_mgr = LicenseManager()
market = NFTMarketplace()

release_date = "2025-03-31"
ip_id = ip_chain.register_asset("Hoa Minzy", "MV: Bắc Bling", "Music", release_date, "Cục Bản quyền")

# Chia sẻ bản quyền
royalty_contract = SmartContractRoyalty(ip_id, {
    "HoaMinzy": 50,
    "XuanHinh": 30,
    "TuanCry": 20
})
royalty_contract.distribute_payment(5000000)

# Cấp License
license_mgr.grant_license("User123", ip_id, "display", "2025-12-31")
license_mgr.check_license("User123", ip_id)

# Bán tác phẩm
market.list_asset(ip_id, 10000000)
market.buy_asset("BuyerX", ip_chain, ip_id)

# In danh sách tác phẩm
ip_chain.print_assets()

[+] Đăng ký thành công: MV: Bắc Bling bởi Hoa Minzy

💸 Phân phối 5000000 VND cho tác phẩm 8bfc025b3ad10a5739b14c835735f5e75a85e00c3002310dcd1703c33065a83b
  -> HoaMinzy nhận 2500000.00 VND (50%)
  -> XuanHinh nhận 1500000.00 VND (30%)
  -> TuanCry nhận 1000000.00 VND (20%)
[+] Cấp quyền display cho User123 tới 2025-12-31 với tác phẩm 8bfc025b3ad10a5739b14c835735f5e75a85e00c3002310dcd1703c33065a83b
✔ User123 có quyền 'display' với 8bfc025b3ad10a5739b14c835735f5e75a85e00c3002310dcd1703c33065a83b tới 2025-12-31
[+] Tác phẩm 8bfc025b3a... đã được niêm yết với giá 10000000 VND
[+] Chuyển quyền cho BuyerX thành công.
💰 BuyerX đã mua tác phẩm với giá 10000000 VND

🔐 Tác phẩm đã đăng ký:
  - MV: Bắc Bling | Chủ: BuyerX | ID: 8bfc025b3a...
